# Question 1

In [1]:
import numpy as np
from scipy.optimize import linprog

np.random.seed(42)

class PricingGame:
    def __init__(self, num_firms=3):
        self.num_firms = num_firms
        self.costs = np.random.uniform(10, 30, num_firms)  # Production costs for each firm
        self.max_price = 100  # Maximum possible price
        self.market_size = 1000  # Total market size
        
        # Price sensitivity coefficients (randomly generated)
        self.price_sensitivity = np.random.uniform(0.5, 1.5, (num_firms, num_firms))
        np.fill_diagonal(self.price_sensitivity, np.random.uniform(1.5, 2.5, num_firms))
    
    def demand(self, prices):
        """Calculate demand for each firm given the prices"""
        base_demand = self.market_size / self.num_firms
        demands = np.full(self.num_firms, base_demand)
        for i in range(self.num_firms):
            for j in range(self.num_firms):
                if i != j:
                    demands[i] += self.price_sensitivity[i, j] * (prices[j] - prices[i])
        return np.maximum(demands, 0)  # Demand cannot be negative
    
    def profit(self, prices):
        """Calculate profit for each firm given the prices"""
        demands = self.demand(prices)
        return (prices - self.costs) * demands
    
    def best_response(self, other_prices, firm_index):
        """Find the best response price for a given firm"""
        def objective(x):
            prices = np.copy(other_prices)
            prices[firm_index] = x[0]
            return -self.profit(prices)[firm_index]  # Negative for maximization
        
        res = linprog(
            c=[1],
            A_ub=[[-1], [1]],
            b_ub=[-self.costs[firm_index], self.max_price],
            method='highs'
        )
        return res.x[0]
    
    def find_nash_equilibrium(self, max_iterations=100, tolerance=1e-6):
        """Find the Nash equilibrium prices using iterative best response"""
        prices = np.random.uniform(self.costs, self.max_price, self.num_firms)
        for _ in range(max_iterations):
            old_prices = np.copy(prices)
            for i in range(self.num_firms):
                other_prices = np.copy(prices)
                prices[i] = self.best_response(other_prices, i)
            if np.all(np.abs(prices - old_prices) < tolerance):
                break
        return prices


game = PricingGame()
nash_prices = game.find_nash_equilibrium()



Firm costs:

Firm 1: 25.34

Firm 2: 15.04

Firm 3: 27.94

Nash equilibrium prices:

Firm 1: 52.45

Firm 2: 45.69

Firm 3: 54.22

Demand at equilibrium:

Firm 1: 320.12

Firm 2: 403.98

Firm 3: 275.89

Profit at equilibrium:

Firm 1: 8687.56

Firm 2: 12366.57

Firm 3: 7257.26

# Question 2

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Set random seed for reproducibility
np.random.seed(42)

class AdGame:
    def __init__(self):
        self.market_size = 1000000  # Total market size
        self.base_shares = np.array([0.4, 0.35, 0.25])  # Initial market shares
        self.ad_effectiveness = np.array([0.1, 0.12, 0.15])  # Advertising effectiveness coefficients
        self.cost_coefficient = np.array([1.2, 1.0, 0.8])  # Cost coefficients
        self.max_budget = 100000  # Maximum advertising budget

    def market_share(self, ad_spends):
        """Calculate market shares based on advertising spending"""
        weighted_spends = self.ad_effectiveness * ad_spends
        total_effect = np.sum(weighted_spends)
        if total_effect == 0:
            return self.base_shares
        return self.base_shares + (weighted_spends / total_effect) * (1 - np.sum(self.base_shares))

    def profit(self, ad_spends):
        """Calculate profits for each company"""
        shares = self.market_share(ad_spends)
        revenues = shares * self.market_size
        costs = self.cost_coefficient * ad_spends
        return revenues - costs

    def best_response(self, other_spends, company_index):
        """Find the best response advertising spend for a given company"""
        def neg_profit(x):
            spends = np.copy(other_spends)
            spends[company_index] = x[0]
            return -self.profit(spends)[company_index]

        res = minimize(neg_profit, [self.max_budget/2], bounds=[(0, self.max_budget)], method='L-BFGS-B')
        return res.x[0]

    def find_nash_equilibrium(self, max_iterations=100, tolerance=1e-6):
        """Find the Nash equilibrium advertising spends using iterative best response"""
        spends = np.random.uniform(0, self.max_budget, 3)
        for _ in range(max_iterations):
            old_spends = np.copy(spends)
            for i in range(3):
                other_spends = np.copy(spends)
                spends[i] = self.best_response(other_spends, i)
            if np.all(np.abs(spends - old_spends) < tolerance):
                break
        return spends

    def plot_results(self, nash_spends):
        """Plot the results of the game"""
        companies = ['Company A', 'Company B', 'Company C']
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

        # Advertising spending
        ax1.bar(companies, nash_spends)
        ax1.set_title('Nash Equilibrium Advertising Spend')
        ax1.set_ylabel('Advertising Spend')

        # Market share
        shares = self.market_share(nash_spends)
        ax2.pie(shares, labels=companies, autopct='%1.1f%%')
        ax2.set_title('Market Share Distribution')

        # Profit
        profits = self.profit(nash_spends)
        ax3.bar(companies, profits)
        ax3.set_title('Company Profits')
        ax3.set_ylabel('Profit')

        plt.tight_layout()
        plt.show()


game = AdGame()
nash_spends = game.find_nash_equilibrium()
shares = game.market_share(nash_spends)
profits = game.profit(nash_spends)




Nash Equilibrium Advertising Spend:

Company A: 48222.41

Company B: 52894.19

Company C: 60170.29

Equilibrium Market Shares:

Company A: 37.44%

Company B: 35.64%

Company C: 26.92%

Equilibrium Profits:

Company A: 316433.39

Company B: 298163.27

Company C: 209902.05
